In [1]:
import numpy as np
import pandas as pd
from scipy.integrate import odeint
from ipywidgets import widgets
from IPython.display import display
import plotly as pt
import plotly.graph_objs as go

## 1. 주요 변수 소개

- $S$: 미감염자 (Susceptible)
- $E$: 노출자 (Exposed)
- $I$: 감염자 (Infected)
- $C$: 중환자 (Critical)
- $R$: 회복자 (Recovered)
- $D$: 사망자 (Dead)
- $Nn = S + E + I + R$: 사망자를 제외한 전체 인구
- $\mu$: 자연 사망률 (0<x<1)
- $\beta$: 감염률(Infection rate)
- $\gamma$: 회복율(Recovery rate) (0<x<1)
- $e$: 노출 후 발병 비율 (0<x<1)
- $t_{inc}$: 잠복기 기간 (incubation)
- $t_{inf}$: 유증상 -> 중증 기간 (infectious)
- $t_{fat}$: 중증 -> 사망 기간 (fatal)
- $t_{rec}$: 중증 -> 치료 기간 (recover)
- $p(I \rightarrow C)$ : 유증상 -> 중증 확률
- $p(C \rightarrow D)$ : 유증상 -> 사망 확률

In [2]:
class SEIRD():
    @classmethod
    def add_method(cls, mtd):
        setattr(cls, mtd.__name__, mtd)
        return mtd
    
    def __init__(self, mu, beta, gamma, e, t_inc, t_inf, t_fat, t_rec, p_ic, p_cd):
        self.mu = mu # 자연 사망률
        self.beta = beta # 감염률 (Infection rate)
        self.gamma = gamma # 회복율 (Recovery rate)
        self.e = e # 노출 후 발병 비율
        self.t_inc = t_inc # 잠복기 기간 (incubation)
        self.t_inf = t_inf # 유증상 -> 중증 기간 (infectious)
        self.t_fat = t_fat # 중증 -> 사망 기간 (fatal)
        self.t_rec = t_rec # 중증 -> 치료 기간 (recover)
        self.p_ic = p_ic # 유증상 -> 중증 확률
        self.p_cd = p_cd # 유증상 -> 사망 확률

$$
\begin{aligned}
\frac{dS}{dt} &= - \frac{\beta S I}{N} - \mu S \\
\\
\frac{dE}{dt} &= \frac{\beta S I}{N} - \frac{e}{t_{inc}} E - \frac{(1 - e)}{t_{inc}} E  - \mu E \\
\\
\frac{dI}{dt} &= \frac{e}{t_{inc}} E - \frac{1}{t_{inf}}p(I \rightarrow C) I - \gamma (1 - p(I \rightarrow C)) I - \mu I \\
\\
\frac{dC}{dt} &= \frac{1}{t_{inf}}p(I \rightarrow C) I - \frac{1}{t_{death}}p(C \rightarrow D) C - \frac{1}{t_{rec}}(1 - p(C \rightarrow D)) C -  \mu C \\
\\
\frac{dR}{dt} &= \frac{(1 - e)}{t_{inc}} E + \gamma (1 - p(I \rightarrow C)) I + \frac{1}{t_{rec}}(1 - p(C \rightarrow D)) C - \mu R \\
\\
\frac{dD}{dt} &= \frac{1}{t_{death}}p(C \rightarrow D) C - \mu D  \\
\end{aligned}
$$

## 2. dS/dt

`dSdt`는 시간에 따른 감염되지 않은 사람 (S)의 수의 변화율을 의미한다.

$
\begin{aligned}
\frac{dS}{dt} &= - \frac{\beta S I}{N} - \mu S \\
\end{aligned}
$

<br>

$\frac{\beta S I}{N}$ : I와 S 사이의 접촉에 의해 E로 빠져나가는 인구의 변화량

$\mu S$ : 감염되지 않은 사람 (S) 에서 자연적으로 발생하는 총 인구의 변화량

In [ ]:
# dSdt = - (self.beta * S * I) / Nn - self.mu * S

## 3. dE/dt

`dEdt`는 시간에 따라 전염병에 직접적으로 노출된 사람 (E)의 수의 변화율을 의미한다.

$
\begin{aligned}
\frac{dE}{dt} &= \frac{\beta S I}{N} - \frac{e}{t_{inc}} E - \frac{(1 - e)}{t_{inc}} E - \mu E \\
\end{aligned}
$

<br>

$\frac{\beta S I}{N}$ : I와 S 사이의 접촉에 의해 E로 빠져나가는 인구의 변화량

$\frac{e}{t_{inc}} E$ : E에서 I로 이월되는 전염병에 감염된 인원.

$\frac{(1 - e)}{t_{inc}} E $ : 감염되지 않은 인원. 그대로 R 인원으로 이월한다.

$\mu E $ : 노출된 사람 (E) 에서 자연적으로 발생하는 총 인구의 변화량

In [ ]:
# dEdt = (self.beta * S * I) / Nn - (self.e / self.t_inc) * E - ((1 - self.e) / self.t_inc) * E - self.mu * E

## 4. dI/dt

`dI/dt`는 시간에 따라 감염자 수 (I)의 변화율을 의미한다.

$
\begin{aligned}
\frac{dI}{dt} &= \frac{e}{t_{inc}} E - \frac{1}{t_{inf}}p(I \rightarrow C) I - \gamma (1 - p(I \rightarrow C)) I - \mu I \\
\end{aligned}
$

<br>

$\frac{e}{t_{inc}} E$ : E에서 I로 이월되는 전염병에 감염된 인원.

$\frac{1}{t_{inf}}p(I \rightarrow C) I$ : I에서 C로 이월되는 중증 환자

$\gamma (1 - p(I \rightarrow C)) I$ : I에서 R로 이월되는 회복한 환자

$\mu I$ :감염된 인구 (I) 에서 자연적으로 발생하는 총 인구의 변화량

In [ ]:
# dIdt = (self.e / self.t_inc) * E - (1 / self.t_inf) * p_ic * I - self.gamma * (1 - p_ic) * I - self.mu * I

## 5. dC/dt

`dC/dt`는 시간에 따라 중증 환자의 수 (R)의 변화율을 의미한다.

$
\begin{aligned}
\frac{dC}{dt} &= \frac{1}{t_{inf}}p(I \rightarrow C) I - \frac{1}{t_{death}}p(C \rightarrow D) C - \frac{1}{t_{rec}}(1 - p(C \rightarrow D)) C -  \mu C \\
\end{aligned}
$

<br>

$\frac{1}{t_{inf}}p(I \rightarrow C) I$ :I에서 C로 이월되는 중증 환자

$\frac{1}{t_{death}}p(C \rightarrow D) C$ : C에서 D로 이월되는 사망한 환자 

$\frac{1}{t_{rec}}(1 - p(C \rightarrow D)) C$ : C에서 R로 이월되는 중증에서 회복한 환자

$\mu R$ :회복하는 인원 (R) 에서 자연적으로 발생하는 총 인구의 변화량

In [ ]:
# dCdt = (1 / self.t_inf) * p_ic * I - (1 / self.t_fat) * p_cd * C - (1 / self.t_rec) * (1 - p_cd) * C - self.mu * C

## 6. dR/dt

`dR/dt`는 시간에 따라 완치자 수 (R)의 변화율을 의미한다.

$
\begin{aligned}
\frac{dR}{dt} &= \frac{(1 - e)}{t_{inc}} E + \gamma (1 - p(I \rightarrow C)) I + \frac{1}{t_{rec}}(1 - p(C \rightarrow D)) C - \mu R \\
\end{aligned}
$

<br>

$\frac{(1 - e)}{t_{inc}} E$ :감염되지 않은 인원. 그대로 R 인원으로 이월한다

$\gamma (1 - p(I \rightarrow C)) I$ : I에서 R로 이월되는 회복한 환자 

$\frac{1}{t_{rec}}(1 - p(C \rightarrow D)) C$ : C에서 R로 이월되는 중증에서 회복한 환자

$\mu R$ :회복하는 인원 (R) 에서 자연적으로 발생하는 총 인구의 변화량

In [ ]:
# dRdt = ((1 - self.e) / self.t_inc) * E + self.gamma * (1 - p_ic) * I + (1 / self.t_rec) * (1 - p_cd) * C  - self.mu * R

## 5. dD/dt

`dD/dt`는 시간에 따라 사망자 수 (D)의 변화율을 의미한다.

$
\begin{aligned}
\frac{dD}{dt} &= \frac{1}{t_{death}}p(C \rightarrow D) C - \mu D  \\
\end{aligned}
$

<br>

$\frac{1}{t_{death}}p(C \rightarrow D) C$ : C에서 D로 이월되는 사망한 환자

$\mu D$ :사망하는 인원 (D) 에서 자연적으로 발생하는 총 인구의 변화량

In [ ]:
# dDdt = (1 / self.t_fat) * p_cd * C - self.mu * D

In [3]:
@SEIRD.add_method
def deriv(self, y_t, t):
    S, E, I, C, R, D = y_t
    
    # Nn: 사망자를 제외한 전체 인구 수
    Nn = S + E + I + R
    
    # dSdt: 미감염자(S)의 변화율
    dSdt = - (self.beta * S * I) / Nn - self.mu * S
    
    # dEdt: 노출자(E)의 변화율
    dEdt = (self.beta * S * I) / Nn - (self.e / self.t_inc) * E - ((1 - self.e) / self.t_inc) * E - self.mu * E
    
    # dIdt: 감염자(I)의 변화율
    dIdt = (self.e / self.t_inc) * E - (1 / self.t_inf) * p_ic * I - self.gamma * (1 - p_ic) * I - self.mu * I
    
    # dCdt : 중증환자(C)의 변화율
    dCdt = (1 / self.t_inf) * p_ic * I - (1 / self.t_fat) * p_cd * C - (1 / self.t_rec) * (1 - p_cd) * C - self.mu * C
    
    # dRdt: 회복자(R)의 변화율
    dRdt = ((1 - self.e) / self.t_inc) * E + self.gamma * (1 - p_ic) * I + (1 / self.t_rec) * (1 - p_cd) * C  - self.mu * R
    
    # dDdt: 사망자(D)의 변화율
    dDdt = (1 / self.t_fat) * p_cd * C - self.mu * D
        
    return [dSdt, dEdt, dIdt, dCdt, dRdt, dDdt]

In [4]:
@SEIRD.add_method
def get_data_from_csv(country):

    inf_csv = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
    inf = pd.read_csv(inf_csv)
    death_csv = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
    death = pd.read_csv(death_csv)

    inf = inf.replace(np.nan, '', regex=True)
    inf = inf.groupby('Country/Region').sum().reset_index()
    inf = inf.transpose()
    header = inf.iloc[0]
    inf = inf[1:]
    inf.columns = header
    
    death = death.replace(np.nan, '', regex=True)
    death = death.groupby('Country/Region').sum().reset_index()
    death = death.transpose()[1:]
    death.columns = header
    
    return [inf.loc[:, country ], death.loc[:, country ]]

In [7]:
@SEIRD.add_method
def plotly_graph(N, mu, beta, gamma, e, t_inc, t_inf, t_fat, t_rec, p_ic, p_cd,
                   model, y_initial, t_domain, confirmed, deaths):
    
    Solve = odeint(model.deriv, y_initial, t_domain)
    
    S, E, I, C, R, D = [Solve[:,i] for i in range(6)]

    beta_slider  = widgets.FloatSlider(
        value = 1.0,
        min = 0.0,
        max = 2.0,
        step = 0.01,
        description = '𝛽',
        continuous_update = False)
    
    layout = go.Layout(title =  dict(text ='Cases',font =dict(family='Sherif',size=18,color = 'white')),
        yaxis=dict(title="number of cases", zeroline=False,showgrid = True,gridcolor='gray'),
        xaxis=dict(title="DateTime",zeroline = False,showgrid = True, tickangle = 60,gridcolor='gray'),
        paper_bgcolor='#57565e',plot_bgcolor='rgba(0,0,0,0)',font = dict(color = 'lightyellow'))
    
    fig = go.FigureWidget(data=[
        go.Scatter(name="Susciptible", x=t_domain,y=S,line=dict(color="#ff2a17"),mode="lines"),
        go.Scatter(name="Exposed", x=t_domain,y=E,line=dict(color="blue"),mode="lines"),
        go.Scatter(name='Infected', x=t_domain,y=I,line=dict(color="black"), mode="lines"),
        go.Scatter(name='Critical', x=t_domain,y=C,line=dict(color="purple"), mode="lines"),
        go.Scatter(name='Recovered', x=t_domain, y=R,line=dict(color="green"), mode="lines"),
        go.Scatter(name='Deaths', x=t_domain,y=D,line=dict(color="yellow"), mode="lines"),
        go.Scatter(name='Deaths_data', x=np.arange(len(deaths.values)),
            y=deaths.values,line=dict(color="darkgray"), mode="lines"),
        go.Scatter(name='Infected_data', x=np.arange(len(confirmed.values)),
            y=confirmed.values,line=dict(color="blueviolet"), mode="lines")],layout=layout)

    
    fig.layout.update(barmode='group')
    
    def response(change):
        with fig.batch_update():
            
            model.beta = beta_slider.value
            
            model.gamma = 1/14
            model.mu = 0.01 / 365
            model.e = 0.5
            model.t_inc = 5.
            model.t_inf = 10.
            model.t_fat = 7.
            model.t_rec = 12.
            model.p_ic = 0.1
            model.p_cd = 0.2
            fig.data[0].y=odeint(model.deriv, y_initial, t_domain)[:,0] #Sus
            fig.data[1].y=odeint(model.deriv, y_initial, t_domain)[:,1] #Sus
            fig.data[2].y=odeint(model.deriv, y_initial, t_domain)[:,2] #Inf
            fig.data[3].y=odeint(model.deriv, y_initial, t_domain)[:,3] #Rec
            fig.data[4].y=odeint(model.deriv, y_initial, t_domain)[:,4] #Deaths
            fig.data[5].y=odeint(model.deriv, y_initial, t_domain)[:,5]
            
    beta_slider.observe(response, names="value")

    #stack the sliders horizontally
    container = widgets.HBox([beta_slider])
    display(container,fig)
    response("any kinda shit for triggering the event") 

In [8]:
if __name__ == "__main__":
    conf_cases, deaths = get_data_from_csv(country = "Korea, South")

    N = 51780579  # Total number of population(uninfected)
    gamma = 1/14
    beta = 1.0
    mu = 0.01/365
    e = 0.5
    t_inc = 5.
    t_inf = 10.
    t_fat = 7.
    t_rec = 12.
    p_ic = 0.1
    p_cd = 0.2
    y_initial = [N, 0, 1, 0, 0, 0]
    time = 1150
    t_domain  = np.linspace(0, time, time)
    
    model = SEIRD(mu, beta, gamma, e, t_inc, t_inf, t_fat, t_rec, p_ic, p_cd)
    plotly_graph(N, mu, beta, gamma, e, t_inc, t_inf, t_fat, t_rec, p_ic, p_cd,
                   model,y_initial,t_domain,conf_cases, deaths)

FigureWidget({
    'data': [{'line': {'color': '#ff2a17'},
              'mode': 'lines',
              'name'…